### GPT-2 학습해보기

In [1]:
!pip install transformers==4.30

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [2]:
import torch
torch.cuda.is_available()

False

In [5]:
!pip install gensim

In [6]:
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1V4rTx4yaAg0x1NY1MpNRY2Dp1nKeyOQ7" -o wiki_20190620_small.txt

awk: cannot open ./cookie (No such file or directory)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 75.1M  100 75.1M    0     0  11.7M      0  0:00:06  0:00:06 --:--:-- 18.3M


In [7]:
path = '/content/wiki_20190620_small.txt'

지금까지는 BertWordPieceTokenizer를 사용해왔다면, <br>
이번에는 SentencePiceBPETokenizer를 사용해 모델을 학습해보겠습니다. <br>

In [8]:
# 내부적인 알고리즘은 BPE 로 동일하고 tokenizer 의 경우에는 띄어쓰기 부분에 ‘_’ 기호를 추가하는 과정만 다르다고 생각하면 됨
from tokenizers import SentencePieceBPETokenizer
from tokenizers.normalizers import BertNormalizer

tokenizer = SentencePieceBPETokenizer()

tokenizer._tokenizer.normalizer = BertNormalizer(clean_text=True,
handle_chinese_chars=False,
lowercase=False)

# 마찬가지로 tokenizer 학습
# 학습데이터와 vocab_size 를 지정
tokenizer.train(
    path,
    vocab_size=10000,
    special_tokens=[
        "<s>",
        "<pad>",
        "</s>",
        "<unk>",
    ],
)

BERT 와 다른점이 하나 있는데 BERT 는 학습할 때 앞에 [CLS] 토큰과 뒤에 [SEP] 토큰을 부착하게 되어있었음

그렇기 때문에 special token 에 [PAD], [CLS] [SEP], [UNK] 들이 의무적으로 들어갔었음

하지만 GPT2 가 학습하는 과정을 상상해보면 앞에 토큰이 나왔을 때 계속해서 다음 토큰을 예측하는 방식으로 이루어지고 있음

그래서 [CLS] 토큰이나 [SEP] 토큰이 필요하지 않음

하지만 이 경우에 fasttext 도 상상해보면 음절의 시작과 긑을 표시해주는 “<”, “>” 기호 넣었음

역시 GPT2 도 문장의 끝과 시작을 표시해주는 special token 을 부착해줄 수 있음

그래서 나중에 생성할 때 문장단위로 생성하도록 control 할 수 있음

special token 에 문장의 시작을 의미하는 < s > 와 문장의 끝을 의미하는 < /s> 를 추가해줄 수 있음

tokenizer 를 학습했으니 test 를 해보자.

In [9]:
print(tokenizer.encode("이순신은 조선 중기의 무신이다."))
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").ids)
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))

# 문장에서 띄어쓰기가 들어가는 부분 즉, 어절의 시작부분에는 ‘_’ 기호가 부착이됨
# SentencePiece를 사용하면, 나중에 decoding 과정에서 '_' 만 ' '로 replace해주면 띄어쓰기 복원이 가능해집니다.

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[6284, 693, 1246, 6180, 698, 6057, 699, 262, 13]
['▁이순신', '은', '▁조선', '▁중기', '의', '▁무신', '이', '다', '.']
이순신은 조선 중기의 무신이다.


In [10]:
# tokniezr 를 저장할 땐 아래 명령어를 사용
tokenizer.save_model(".")
# 나만의 tokenizer 가 만들어졌음

['./vocab.json', './merges.txt']

In [11]:
tokenizer = SentencePieceBPETokenizer.from_file(vocab_filename="vocab.json", merges_filename="merges.txt")
# from_file() 함수를 사용하면 내가 학습한 모델을 그대로 불러와서 tokenizer 로서 사용할 수 있음

In [12]:
print(tokenizer.encode("이순신은 조선 중기의 무신이다."))
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").ids)
print(tokenizer.encode("이순신은 조선 중기의 무신이다.").tokens)
print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))

# 불러와서 test 해보면 동일한 결과를 확인가능
# 우리가 원하는 것은 문장의 시작과 끝에 <s>, </s> 를 부착함으로써 문장을 구분해내는 것을 원하고 있음

# 그런데 이렇게 다 분리되서 나옴
# 이유는 special token 으로 지정이 되어있지 않음

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[6284, 693, 1246, 6180, 698, 6057, 699, 262, 13]
['▁이순신', '은', '▁조선', '▁중기', '의', '▁무신', '이', '다', '.']
['▁<', 's', '>', '이', '순', '신', '은', '▁조선', '▁중기', '의', '▁무신', '이', '다', '.', '<', '/', 's', '>']
<s>이순신은 조선 중기의 무신이다.</s>


In [13]:
# 의도적으로 tokenizer 에 add_special_tokens 를 통해 명시적으로 알려줘야 함

tokenizer.add_special_tokens(["<s>", "</s>", "<unk>", "<pad>", "<shkim>"])
tokenizer.pad_token_id = tokenizer.token_to_id("<pad>")
tokenizer.unk_token_id = tokenizer.token_to_id("<unk>")
tokenizer.bos_token_id = tokenizer.token_to_id("<bos>")
tokenizer.eos_token_id = tokenizer.token_to_id("<eos>")

print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids)
print(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").tokens)
print(tokenizer.decode(tokenizer.encode("<s>이순신은 조선 중기의 무신이다.</s>").ids, skip_special_tokens=True))

# 이렇게 등록하고 나면 문장내에 스페셜 토큰을 부착하게 되더라도 올바르게 tokenizing 된 것은 확인할 수 있음

[0, 6284, 693, 1246, 6180, 698, 6057, 699, 262, 13, 2]
['<s>', '▁이순신', '은', '▁조선', '▁중기', '의', '▁무신', '이', '다', '.', '</s>']
이순신은 조선 중기의 무신이다.


In [14]:
# 본격적으로 GPT2 를 학습
from transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(
  vocab_size=tokenizer.get_vocab_size(),
  bos_token_id=tokenizer.token_to_id("<s>"),
  eos_token_id=tokenizer.token_to_id("</s>"),
)
# 모델생성
model = GPT2LMHeadModel(config)

# transformers 에서 BERT 와 마찬가지로 GPT2 에 대해서도 학습을 제공해줌
# GPT2 에 대한 껍데기 모델을 만들어야 함
# 항상 vocab_size 를 우리가 만든 tokenizer 의 vocab_size 로 명확하게 지정해줘야 함
# bos_token(문장의 시작을 알려주는 토큰) 과 eos_token(문장의 끝을 알려주는 토큰)을 어떤 것으로 정의했는지 명확하게 저장할 수 있음
# 다양한 옵션들이 많아서 layer size 를 줄일 수도 있고 내부의 embedding size 도 줄일 수 있고 다양한 설정을 할 수 있음

In [15]:
model.num_parameters() # 파라미터 개수도 알 수 있음

93523200

In [19]:
# GPT2 에 넣어줄 데이터셋도 만들어야 함
import json
import os
import pickle
import random
import time
import warnings
from typing import Dict, List, Optional

import torch
from torch.utils.data.dataset import Dataset

from filelock import FileLock

from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.utils import logging

logger = logging.get_logger(__name__)

class TextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        block_size = block_size - tokenizer.num_special_tokens_to_add(is_pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")
                # 여기서부터 본격적으로 데이터셋을 만들기 시작
                self.examples = []
                text = ""
                with open(file_path, encoding="utf-8") as f: #  file_path(wiki small 데이터)
                    lines = f.readlines()
                    for line in lines:
                        line = line.strip()
                        line = "<s>"+line+"</s>" # 학습 데이터 앞 뒤에 문장 구분 기호를 추가
                        text += line    # 'text' 객체에 모든 학습 데이터를 다 합쳐버림
                tokenized_text = tokenizer.encode(text).ids

                # 모델의 최대 sequence length만큼 데이터를 잘라서 저장
                for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                    self.examples.append(
                        tokenized_text[i : i + block_size]
                    )

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

GPT2 모델은 BERT 학습보다 훨씬 간단함

mask 의 대한 개념도 없었고 Next Sentence Prediction 도 필요하지 않음

그냥 우리가 학습해야하는 sentence 를 통채로 주면 GPT 모델이 이 토큰 다음에 나올 수 있는 토큰은 이거다 다음은 이거다 라는 것을 순차적으로 자동으로 학습하게됨

그래서 sentence 만 만들어줘서 GPT 모델에 넘겨주면 됨

캐싱 관련된 부분을 지나서 file_path(wiki small 데이터) 를 한줄씩 읽어서 strip() 해주고 의도적으로 line 앞뒤에 < s >, < /s > 를 넣어주게 됨 그리고 text 라는 하나의 변수에 모든 학습데이터를 통채로 넣게 됨

text 안에 기존의 wiki 데이터와 동일하지만 모든 문장마다 < s >, < /s> 이 부착된 형태로 text 가 만들어짐

그 만들어진 것을 tokenizing 하면 tokenizer 에 따라서 vocab_id 로 변환된 데이터가 tokenized_text 로 입력으로 들어가게 됨

그러면 내가 가진 모델의 sequence length 가 있을 건데 BERT 는 512 토큰이 있었고 GPT2 도 그런 block_size 를 가질텐데 tokenized_text 데이터는 전체 wiki 의 데이터를 다가지고 있으니 이거를 block_size 만큼으로 쪼개게 됨

toknized_text[i : i + bolck_size] 로 쪼개서 exmaples 에 넣게 됨

다 넣게되면 우리가 원하는 학습데이터가 완성됨

In [20]:
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=path,
    block_size=128,
)
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(    # GPT는 생성모델이기 때문에 [MASK] 가 필요 없다
    tokenizer=tokenizer, mlm=False, # mask를 의미하는 것을 False로
)
# TextDataset 에 tokenizer, file_path, block_size 넣고 만들어 줌
# 실습을 위해 block_size 를 128 로 줬지만 원래 사이즈는 1024

이렇게 데이터셋인 밥이 만들어지면 떠먹여주는 데이터로더가 필요함

transformers 에서 DataCollatorForLanguageModeling 이라는 함수를 제공해주고 있음

이전에는 data_collator 를 mask 를 지원해주는 기능을 넣었었는데 이번에는 mask 가 필요없음

그래서 LanguageModeling 을 위한 함수를 불러왔음

In [21]:
# 이 함수를 call 할 때 mlm 은 mask 를 의미하는 건데 GPT2 는 mask 가 필요없으므로 False 로 두고 데이터셋을 만들면
print(dataset[0])
# 학습을 위한 데이터셋이 만들어지게 됨

tensor([   0, 4291, 1911, 1076, 1021,  456, 1076, 1245,  874, 1050, 5751, 1009,
        2694, 1808, 1197, 3942, 1182, 1390, 1340, 1005, 1004, 1007,    2,    0,
        1021,  456, 1245,  874, 1009, 5000,  746, 1489,  874, 6691, 6076,  701,
         579, 2442, 1025, 2295, 1004, 1007,    2,    0, 5000, 8593, 1758, 1016,
        2129, 1011, 1039, 1004, 1007,    2,    0, 1029, 1103, 2539, 1006, 2546,
        8277, 1138, 7604, 1138, 5319,  943, 1008, 1271, 3629, 1020, 1036, 1011,
        1039, 1004, 1007,    2,    0, 4825, 1024, 1197, 2539, 7387, 1022, 1178,
         916, 1011, 1039, 1017, 1169, 2120,  845, 1138, 9501, 1071, 1012, 1014,
           3, 1162, 1013, 2407, 1012, 1615, 1030, 1004, 1007,    2,    0, 1029,
        1008, 4572, 1005, 1076, 2120,  845, 5414, 1076, 1022, 1441, 1374, 1004,
        1007,    2,    0, 4288, 1024, 5000, 1050, 4229])


In [23]:
# 동일하게 TrainArgument 에 우리가 원하는 argument 를 채워주게 됨

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=64, # 512:32  # 128:64
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
trainer.train()
# BERT 와 마찬가지로 굉장히 쉽게 GPT2 도 학습할 수 있음

In [ ]:
trainer.save_model()

In [ ]:
# 학습을 했으니 test
USE_GPU = 1
# Device configuration
device = torch.device('cuda' if (torch.cuda.is_available() and USE_GPU) else 'cpu')

In [ ]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("<s>이순신", add_special_tokens=True).ids).unsqueeze(0).to('cuda')

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

테스트를 할 때는 GPT2 모델 같은 경우에 generate 라는 특별한 함수를 제공해주고 있음

다음 실습에서는 generate 함수에 대한 분석을 보여주겠음

간단히 설명하자면 입력값이 들어가고 GPT2 가 단어를 생성하기에 앞서서 앞에 무슨 단어를 시작으로 할 것이냐를 input_ids 로 넣게 됨

input_ids 이기 때문에 당연히 tokenize 된 vocab_id 가 들어가게 됨

do_sample=True 면 sample 을 몇개 반환하라는 의미